# C.V. Agents
In which CrewAI agents are used to optimize my C.V. for a particular job posting.

## These are the Things

### The Data Flow

1. Job posting URL → Job Analyst → Structured job requirements
2. Candidate data + Job requirements → Candidate Profiler → Profile analysis  
3. Profile + Job requirements → CV Strategist → Optimization recommendations
4. Optimized data → Template engine → LaTeX → PDF

*Imports needed just about everywhere:*

In [ ]:
from config import get_config
from crewai import Agent, Task, LLM
from crewai_tools import SerperDevTool, ScrapeWebsiteTool

*..as well as the config:*

In [ ]:
config = get_config()


### The Job Analyst

In [ ]:
from crewai_tools import SerperDevTool, ScrapeWebsiteTool

search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()

llm = LLM(
    model=config.job_analyst_model,
    temperature=float(config.job_analyst_temperature),
)

job_analyst = Agent(
    role="Tech Job Researcher",
    goal="Make sure to do amazing analysis on job posting to help job applicants",
    tools=[scrape_tool, search_tool],
    verbose=True,
    llm=llm,
    backstory=(
        "As a Job Researcher, your prowess in "
        "navigating and extracting critical "
        "information from job postings is unmatched."
        "Your skills help pinpoint the necessary "
        "qualifications and skills sought "
        "by employers, forming the foundation for "
        "effective application tailoring."
    ),
)

*I'll build the inputs as they become apparent:*

In [ ]:
inputs = {
    "job_posting_url": "https://app.welcometothejungle.com/dashboard/jobs/oA1SArxV",
}

### The Job Analysis Task

In [ ]:
from optimizer.models import JobPosting

job_analysis_task = Task(
    description=(
        "Analyze the job posting URL provided ({job_posting_url}) to extract and structure "
        "the following information:\n"
        "- Job title and company name\n"
        "- Industry sector\n"
        "- Complete job description\n"
        "- Experience level required (entry, mid, senior, etc.)\n"
        "- General requirements (education, certifications, years of experience)\n"
        "- Required skills (must-have technical and soft skills)\n"
        "- Preferred skills (nice-to-have skills that give candidates an advantage)\n"
        "- Key responsibilities and day-to-day activities\n"
        "Use the scraping and search tools to gather comprehensive information."
    ),
    expected_output=(
        "A structured JobPosting object containing title, company, industry, "
        "description, experience_level, requirements, required_skills, "
        "preferred_skills, and responsibilities extracted from the job posting."
    ),
    output_pydantic=JobPosting,
    agent=job_analyst,
)

In [ ]:
# Create the vectorstore for the knowledge base
from embedding_tool import KnowledgeBaseEmbedder
embedder = KnowledgeBaseEmbedder()
result = embedder.create_embeddings()